# 라이브러리 호출 및 access key

In [ ]:
import requests
import json
import pprint
import pandas as pd
from itertools import islice

In [ ]:
flight_access_key = '***'
price_access_key = '***'
price_secret_key = '***'

# real-time flights

https://aviationstack.com/documentation

In [ ]:
url = 'http://api.aviationstack.com/v1/flights'
params ={'access_key' : flight_access_key, 'limit':limit, 'dep_iata':'ICN'}

response = requests.get(url, params=params)
flights = json.loads(response.text)

flight_list = []

try:
    for flight in flights.get('data'):
        arrival_country = flight['arrival']['iata']               # 도착 국가
        arrival_airport = flight['arrival']['airport']            # 도착 공항
        arrival_sched_time  = flight['arrival']['scheduled']      # 도착 예정 시간
        departure_airport = flight['departure']['airport']        # 출발 공항 (인천공항)
        departure_sched_time = flight['departure']['scheduled']   # 출발 예정 시간
        flight_iata = flight['flight']['iata']                    # 항공편명

        flight_dict = {'flight_iata':flight_iata, 'departure_sched_time':departure_sched_time, 'departure_airport':departure_airport, \
                    'arrival_country':arrival_country, 'arrival_airport':arrival_airport, 'arrival_sched_time':arrival_sched_time}
        flight_list.append(flight_dict)
except:
    print(response.status_code)
    print(response.text)

flight_list[:3]

[{'flight_iata': 'AF267',
  'departure_sched_time': '2024-01-11T11:45:00+00:00',
  'departure_airport': 'Seoul (Incheon)',
  'arrival_country': 'CDG',
  'arrival_airport': 'Charles De Gaulle',
  'arrival_sched_time': '2024-01-11T18:25:00+00:00'},
 {'flight_iata': 'LJ211',
  'departure_sched_time': '2024-01-11T11:45:00+00:00',
  'departure_airport': 'Seoul (Incheon)',
  'arrival_country': 'NRT',
  'arrival_airport': 'Narita International Airport',
  'arrival_sched_time': '2024-01-11T14:15:00+00:00'},
 {'flight_iata': 'MU2990',
  'departure_sched_time': '2024-01-11T11:45:00+00:00',
  'departure_airport': 'Seoul (Incheon)',
  'arrival_country': 'WUX',
  'arrival_airport': 'Wuxi',
  'arrival_sched_time': '2024-01-11T12:45:00+00:00'}]

# flight offers search (price)

https://developers.amadeus.com/self-service/category/flights/api-doc/flight-offers-search



## token

In [ ]:
!curl "https://test.api.amadeus.com/v1/security/oauth2/token" \
    -H "Content-Type: application/x-www-form-urlencoded" \
    -d f"grant_type=client_credentials&client_id={price_access_key}&client_secret={price_secret_key}"

## api

In [ ]:
token = '***'
url = "https://test.api.amadeus.com/v2/shopping/flight-offers"

headers = {
    'Authorization': f'Bearer {token}',
    'Content-Type': 'application/x-www-form-urlencoded'
}

price_list = []


for f in flight_list:
    arrival_country = f['arrival_country']
    params = {
        'originLocationCode': 'ICN',
        'destinationLocationCode': arrival_country,
        'departureDate': '2024-01-11',
        'currencyCode': 'KRW',
        'adults': '1',
        'max': 10,
    }

    response = requests.get(url, headers=headers, params=params)

    try:
        offers = response.json()
        for offer in offers.get('data'):
            first_flight = offer.get('itineraries')[0]['segments'][0]
            flight_iata = first_flight['carrierCode'] + first_flight['number']
            departure_sched_time = first_flight['departure']['at']
            price = offer['price']['total']
            cabin = offer['travelerPricings'][0]['fareDetailsBySegment'][0]['cabin']

            price_dict = {'flight_iata':flight_iata, 'departure_sched_time':departure_sched_time,\
                            'price':price, 'cabin':cabin}
            price_list.append(price_dict)
    except Exception as e:
        print(e)
        print(response.status_code)
        print(response.text)

In [ ]:
pprint.pprint(price_list[:3])
print(len(price_list))

[{'cabin': 'ECONOMY',
  'departure_sched_time': '2024-01-11T09:53:00',
  'flight_iata': '6X957',
  'price': '128000.00'},
 {'cabin': 'ECONOMY',
  'departure_sched_time': '2024-01-11T14:05:00',
  'flight_iata': '6X956',
  'price': '128000.00'},
 {'cabin': 'ECONOMY',
  'departure_sched_time': '2024-01-11T18:45:00',
  'flight_iata': 'KE705',
  'price': '155600.00'}]
978
